In [222]:
from confluent_kafka import Consumer
from datetime import datetime
import uuid
import os
from dotenv import load_dotenv
import snowflake.connector
import os
import json
import re
from statistics import mean
load_dotenv()

KAFKA_SERVER = os.getenv('KAFKA_SERVER')
KAFKA_USERNAME=os.getenv('KAFKA_USERNAME')
KAFKA_PASSWORD=os.getenv('KAFKA_PASSWORD')
KAFKA_TOPIC_NAME = os.getenv('KAFKA_TOPIC_NAME')

values = []

c = Consumer({
    'bootstrap.servers': KAFKA_SERVER,
    'group.id': f'deleton' +str(uuid.uuid1()),
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': KAFKA_USERNAME,
    'sasl.password': KAFKA_PASSWORD,
    'session.timeout.ms': 6000,
    'heartbeat.interval.ms': 1000,
    'fetch.wait.max.ms': 6000,
    'auto.offset.reset': 'latest',
    'enable.auto.commit': 'false',
    'max.poll.interval.ms': '86400000',
    'topic.metadata.refresh.interval.ms': "-1",
    "client.id": 'id-002-005',
})

c.subscribe([KAFKA_TOPIC_NAME])
"""Kafka consumer ready and consumes messages"""

'Kafka consumer ready and consumes messages'

In [223]:
def extract_values_from_log(string):
    regex=r'\d+.?\d+|\d'
    log_values = re.findall(regex,string)
    return log_values

In [236]:
def wait_for_system_log():
    while True:
        kafka_message = c.poll(0.5)
        if kafka_message is not None:
            log = kafka_message.value().decode('utf-8')

            if 'SYSTEM' in log:
                return kafka_message

In [237]:
def convert_unix_to_date(unix_timestamp):
    unix_timestamp /= 1000  #convert to seconds as unix timestamp is in milliseconds
    converted_to_date = datetime.utcfromtimestamp(unix_timestamp) 
  
    current_time = datetime.now()
    if converted_to_date > current_time:
        return 'N/A'

    return converted_to_date.strftime('%Y-%m-%d')

In [238]:
def split_full_name(name):
    name_list = name.split(' ')
    titles = ['Mr','Mrs','Miss','Ms','Dr']

    if name_list[0] in titles:
        first_name = name_list[1]
        try:
            last_name = name_list[2]
        except IndexError:
            last_name = None

    else:
        first_name = name_list[0]
        last_name = name_list[1]

    return first_name,last_name

In [239]:
def flatten_list(address_list):
    flat_list = []

    for element in address_list:
        if type(element) is list:
          
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list


def split_address(address):
    address_list = address.split(',')

    if len(address_list) < 4:
        address_list[0] = address_list[0].split(' ', 1) 
        address_list = flatten_list(address_list)

    house_number = address_list[0]
    street_name = address_list[1].title()
    region = address_list[2].title()
    postcode = address_list[3]

    return house_number,street_name,region,postcode

In [240]:
def clean_user_data(user_dictionary):
    
    user_dictionary['account_create_date'] = convert_unix_to_date(user_dictionary['account_create_date'])
    user_dictionary['date_of_birth'] = convert_unix_to_date(user_dictionary['date_of_birth'])

    first_name,last_name = split_full_name(user_dictionary['name'])

    user_dictionary['first_name'] = first_name
    user_dictionary['last_name'] = last_name

    house_number,street_name,region,postcode = split_address(user_dictionary['address'])

    user_dictionary['house_number'] = house_number
    user_dictionary['street_name'] = street_name
    user_dictionary['region'] = region
    user_dictionary['postcode'] = postcode
    user_dictionary['gender'] = user_dictionary['gender'].title()

    return user_dictionary

In [242]:
wait_for_first_user = True #Wait for current user to finish, so can obtain all relevant information
first_user_collected = False #Add check for first user so data is not sent to snowflake prematurely

resistance_list = []
power_list = []
heart_rate_list = []
rpm_list = []

while True:
    kafka_message = c.poll(0.5)

    if wait_for_first_user:
        kafka_message = wait_for_system_log()
        wait_for_first_user = False


    if kafka_message is not None: #exclude none values
        log = kafka_message.value().decode('utf-8')


        if 'SYSTEM' in log:
            first_user_collected = True
            
            system_log = json.loads(log).get('log')
            split_log = system_log.split(' mendoza v9: [SYSTEM] data = ')
            begin_timestamp = split_log[0]
            print(begin_timestamp)
            dictionary_string = split_log[1][:-1]
            user_dictionary = json.loads(dictionary_string)
            
            user_dictionary = clean_user_data(user_dictionary)

            print(user_dictionary)


        elif 'INFO' in log: #only check for strings with INFO

            values = json.loads(log)
            new_log = values.get('log')
         

            if 'Ride' in new_log: #process strings with Ride info
                new_log = new_log.split(' mendoza v9: [INFO]: Ride - ')
               
                log_values = extract_values_from_log(new_log[1])

                duration = int(float(log_values[0]))
                resistance_list.append(int(log_values[1]))
         

            elif 'Telemetry' in new_log:
                new_log = new_log.split(' mendoza v9: [INFO]: Telemetry - ')
               
                log_values = extract_values_from_log(new_log[1])

                heart_rate_list.append(int(log_values[0]))
                rpm_list.append(int(log_values[1]))
                power_list.append(round(float(log_values[2]),3))


        elif '---' in log and first_user_collected: #New user is starting, so load collected data into snowflake
            total_power = sum(power_list)
            mean_power = mean(power_list)
            mean_rpm = mean(rpm_list)
            mean_heart_rate = mean(heart_rate_list)
            mean_resistance = mean(resistance_list)
                


2022-10-05 13:21:25.537016
{'user_id': 4568, 'name': 'Christine Powell', 'gender': 'Female', 'address': 'Studio 28D,Alexander village,Barberland,WC6M 5LH', 'date_of_birth': '1995-04-03', 'email_address': 'christine.powell@gmail.com', 'height_cm': 152, 'weight_kg': 46, 'account_create_date': '2022-02-01', 'bike_serial': 'SN0000', 'original_source': 'google ads', 'first_name': 'Christine', 'last_name': 'Powell', 'house_number': 'Studio 28D', 'street_name': 'Alexander Village', 'region': 'Barberland', 'postcode': 'WC6M 5LH'}


KeyboardInterrupt: 

In [243]:
USER = os.environ.get('USER')
ACCOUNT = os.environ.get('ACCOUNT')
PASSWORD = os.environ.get('PASSWORD')
WAREHOUSE= os.environ.get('WAREHOUSE')
DATABASE= os.environ.get('DATABASE')
SCHEMA= os.environ.get('SCHEMA')

In [244]:
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema='ZOOKEEPERS_BATCH_PRODUCTION'
)
cs = conn.cursor()
"""Already created schema to use"""

'Already created schema to use'

In [279]:
query = "SHOW SCHEMAS"
create_table = """CREATE TABLE IF NOT EXISTS rides (
  ride_id INT DEFAULT seq1.nextval PRIMARY KEY,
  user_id INT,
  begin_timestamp TIMESTAMP NOT NULL,
  total_duration_sec INT NOT NULL,
  total_power FLOAT NOT NULL,
  mean_power FLOAT NOT NULL,
  mean_resistance FLOAT NOT NULL,
  mean_rpm FLOAT NOT NULL,
  mean_heart_rate FLOAT NOT NULL
);"""

alter_table = """ALTER TABLE users
  ADD FOREIGN KEY (user_id) REFERENCES rides(user_id);"""

result = cs.execute(alter_table)
result.fetchmany(100)

ProgrammingError: 001456 (42601): SQL compilation error:
primary or unique key does not exist for the columns referenced by the foreign key